#Homework 06: One-Versus-All Support Vector Classification
## Kerem Aksoy

##Importing libraries

In [ ]:
import cvxopt as cvx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.spatial.distance as dt
from sklearn.metrics import accuracy_score #For evaluating accuracy

##Importing dataset

In [ ]:
data_set_x = np.genfromtxt("hw06_images.csv", delimiter = ",")
data_set_y = np.genfromtxt("hw06_labels.csv",delimiter = ",")

Dividing data set into test set and training set

In [ ]:
x_train = data_set_x[:1000,:] #1000 points for traning
x_test = data_set_x[1000:,:] #4000 points for testing

y_train = data_set_y[:1000].astype(int)
y_test = data_set_y[1000:].astype(int)
# get number of samples and number of features
N_train = len(y_train)
N_test = len(y_test)
D_train = x_train.shape[1] #Take the dimensinality
labels  = np.max(y_train);
print(labels)
print(N_train)
print(N_test)
print(D_train)

5
1000
4000
784


## Distance and Kernel Functions


$\begin{equation}
    \begin{split}
        d(\boldsymbol{x}_{i}, \boldsymbol{x}_{j}) &= ||\boldsymbol{x}_{i} - \boldsymbol{x}_{j}||_{2} = \sqrt{(\boldsymbol{x}_{i} - \boldsymbol{x}_{j})^{\top} (\boldsymbol{x}_{i} - \boldsymbol{x}_{j})} = \sqrt{\sum\limits_{d = 1}^{D}(x_{id} - x_{jd})^{2}} \\
        k(\boldsymbol{x}_{i}, \boldsymbol{x}_{j}) &= \exp\left(-\dfrac{||\boldsymbol{x}_{i} -\boldsymbol{x}_{j}||_{2}^{2}}{2s^{2}}\right)
    \end{split}
\end{equation}$

In [ ]:
# define Gaussian kernel function
#This return NxN matrix like in the lecture notes
def gaussian_kernel(X1, X2, s):
    D = dt.cdist(X1, X2) #Euclidian distance
    K = np.exp(-D**2 / (2 * s**2))
    return(K)

#Learning Algorithm

#### Primal Problem
$\begin{equation}
	\begin{split}
		\mbox{minimize}\;\;& \dfrac{1}{2} ||\boldsymbol{w}||_{2}^{2} + C \sum\limits_{i = 1}^{N} \xi_{i} \\
		\mbox{with respect to}\;\;& \boldsymbol{w} \in \mathbb{R}^{D},\;\; \boldsymbol{\xi} \in \mathbb{R}^{N},\;\; w_{0} \in \mathbb{R} \\
		\mbox{subject to}\;\;& y_{i} (\boldsymbol{w}^{\top} \boldsymbol{x}_{i} + w_{0}) \geq 1 - \xi_{i} \;\;\;\; i = 1,2,\dots,N \\
		& \xi_{i} \geq 0\;\;\;\; i = 1,2,\dots,N \\
		\mbox{where}\;\;& C \in \mathbb{R}_{+}
	\end{split}
\end{equation}$

#### Dual Problem
$\begin{equation}
	\begin{split}
		\mbox{maximize}\;\;& \sum\limits_{i = 1}^{N} \alpha_{i} - \dfrac{1}{2} \sum\limits_{i = 1}^{N} \sum\limits_{j = 1}^{N} \alpha_{i} \alpha_{j} y_{i} y_{j} k(\boldsymbol{x}_{i}, \boldsymbol{x}_{j}) \\
		\mbox{with respect to}\;\;& \boldsymbol{\alpha} \in \mathbb{R}^{N} \\
		\mbox{subject to}\;\;& \sum\limits_{i = 1}^{N} \alpha_{i} y_{i} = 0 \\
		& 0 \leq \alpha_{i} \leq C\;\;\;\; i = 1,2,\dots,N \\
		\mbox{where}\;\;& C \in \mathbb{R}_{+}
	\end{split}
\end{equation}$

#### Dual Problem in Matrix-Vector Form
$\begin{equation}
	\begin{split}
		\mbox{minimize}\;\;&-\boldsymbol{1}^{\top} \boldsymbol{\alpha} + \dfrac{1}{2} \boldsymbol{\alpha}^{\top} ((y y^{\top}) \odot \mathbf{K}) \boldsymbol{\alpha} \\
		\mbox{with respect to}\;\; & \boldsymbol{\alpha} \in \mathbb{R}^{N} \\
		\mbox{subject to}\;\;& \boldsymbol{y}^{\top} \boldsymbol{\alpha} = 0 \\
		& \boldsymbol{0} \leq \boldsymbol{\alpha} \leq C \boldsymbol{1} \\
		\mbox{where}\;\;& C \in \mathbb{R}_{+}
	\end{split}
\end{equation}$

In [ ]:
#calculate Gaussian kernel #We need to calculate gaussian kernel only once since all of them uses same K matrix(same traning data points with different labels). 
s = 10 #When s^2 is large, we get a more smooth function(underfit occurs). It is like the binwidth value h in the kernel smoother.
K_train = gaussian_kernel(x_train, x_train, s)
alpha_lst = []
w0_lst = []
y_train_c_lst = []
for t in range(labels):
  label = t + 1;
  
  ## Before calculating yyk, we need to make the rows +1 if it belongs to class i otherwise -1 (one vs all method)
  y_train_c = ((y_train == label).astype(int))*2 - 1
  yyK = np.matmul(y_train_c[:,None], y_train_c[None,:]) * K_train

  # set learning parameters
  #For C
  #If it is too large, we have a high penalty for non separable points(we punish so hard), and we may store many support vectors and overfit. 
  #If it is too small, we may find too simple solutions that underfit. 
  C = 10
  epsilon = 1e-3

  P = cvx.matrix(yyK)
  q = cvx.matrix(-np.ones((N_train, 1)))
  G = cvx.matrix(np.vstack((-np.eye(N_train), np.eye(N_train))))
  h = cvx.matrix(np.vstack((np.zeros((N_train, 1)), C * np.ones((N_train, 1)))))
  A = cvx.matrix(1.0 * y_train_c[None,:])
  b = cvx.matrix(0.0)
                      
  # use cvxopt library to solve QP problems
  result = cvx.solvers.qp(P, q, G, h, A, b)
  alpha = np.reshape(result["x"], N_train)
  alpha[alpha < C * epsilon] = 0
  alpha[alpha > C * (1 - epsilon)] = C

  # find bias parameter
  support_indices, = np.where(alpha != 0)
  active_indices, = np.where(np.logical_and(alpha != 0, alpha < C))
  w0 = np.mean(y_train_c[active_indices] * (1 - np.matmul(yyK[np.ix_(active_indices, support_indices)], alpha[support_indices])))

  alpha_lst.append(alpha)
  w0_lst.append(w0)
  y_train_c_lst.append(y_train_c) #Adding the y_train with +1 and -1  according to the class we working on

     pcost       dcost       gap    pres   dres
 0:  2.9600e+01 -4.2260e+04  8e+04  4e-01  4e-14
 1:  1.2992e+01 -7.4611e+03  8e+03  2e-02  3e-14
 2: -6.2643e+02 -2.9089e+03  2e+03  4e-03  3e-14
 3: -8.9105e+02 -1.7204e+03  8e+02  1e-03  3e-14
 4: -1.0255e+03 -1.3482e+03  3e+02  2e-04  4e-14
 5: -1.0893e+03 -1.1778e+03  9e+01  2e-05  4e-14
 6: -1.1096e+03 -1.1353e+03  3e+01  3e-07  4e-14
 7: -1.1169e+03 -1.1209e+03  4e+00  4e-14  5e-14
 8: -1.1184e+03 -1.1185e+03  1e-01  1e-15  5e-14
 9: -1.1185e+03 -1.1185e+03  4e-03  3e-14  5e-14
10: -1.1185e+03 -1.1185e+03  7e-05  6e-15  4e-14
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  1.9161e+02 -4.0951e+04  8e+04  4e-01  4e-14
 1:  1.7375e+02 -7.6611e+03  9e+03  2e-02  3e-14
 2: -4.0615e+02 -2.9616e+03  3e+03  6e-03  2e-14
 3: -6.2557e+02 -1.7531e+03  1e+03  2e-03  2e-14
 4: -7.5417e+02 -1.1618e+03  4e+02  4e-04  3e-14
 5: -8.2696e+02 -9.7495e+02  1e+02  2e-14  3e-14
 6: -8.5262e+02 -8.9665e+02  4e+01  4e-14  3e-1

## Calculating Traning Performance of Traning Set

In [ ]:
#Calculating the scores of all classes for each data point in the traning set
train_predicted_lst = []
for t in range(labels):
  train_predicted = np.matmul(K_train, y_train_c_lst[t][:,None] * alpha_lst[t][:,None]) + w0_lst[t]
  train_predicted_lst.append(train_predicted)

Choosing the class which have the maximum score

In [ ]:
train_predicted_matrix = np.hstack(train_predicted_lst) #This returns 1000(N)x5(number of classes)
train_predictions = np.argmax(train_predicted_matrix,axis=1) + 1 #Since the output of argmax would range 0 to 4, we need to add 1

In [ ]:
all_predictions =  np.hstack(predictions_lst)
predictions = np.argmax(train_predicted_matrix,axis=1) # GIVES 0 TO 1
predictions = predictions + 1 

Confusion Matrix For The Test Set

In [ ]:
confusion_matrix = pd.crosstab(np.reshape(train_predictions, N_train), y_train, rownames = ['y_predicted'], colnames = ['y_train'])
print(confusion_matrix)

y_train        1    2    3    4    5
y_predicted                         
1            207    1    0    9    0
2              2  199    1    1    0
3              0    1  204    6    0
4              0    1    4  185    1
5              0    0    0    0  178


# Calculating Traning Performance Of Test Set

In [ ]:
K_test = gaussian_kernel(x_test, x_train, s) # Here, we simply calculate the similarities between test data and training data
print(K_test.shape) # As expected it will find the similarity of a data point with each of the training data(1000). So, 4000 x 1000 
test_predicted_lst = []
for t in range(labels):
  test_predicted = np.matmul(K_test, y_train_c_lst[t][:,None] * alpha_lst[t][:,None]) + w0_lst[t] #Calculating the predictions(result 4000x1)
  test_predicted_lst.append(test_predicted)

(4000, 1000)


Choosing the class which have the maximum score

In [ ]:
test_predicted_matrix = np.hstack([test_predicted_lst[i] for i in range(labels)]) #This returns 4000(N)x5(number of classes)
test_predictions =  np.argmax(test_predicted_matrix,axis=1) + 1 #Since the output of argmax would range 0 to 4, we need to add 1

Confusion Matrix For Test Set

In [ ]:
confusion_matrix = pd.crosstab(np.reshape(test_predictions, N_test), y_test, rownames = ['y_predicted'], colnames = ['y_test'])
print(confusion_matrix)

y_test         1    2    3    4    5
y_predicted                         
1            641   23    3  137    9
2             43  714   27   40    4
3              4   39  666   90   10
4            100   32   69  541   16
5             12    2    6   15  757


# Part 7
##Learning the algorith with C=0.1,10,100,1000 and S=1 **** Drawing the classification accuracy for each of them

##**Classification Accuracy** = Number of Correct Predictions / Total Predictions

In [ ]:
# accuracy_score(predictions,y_test) calculates accuracy which is imported from sklearn.metrics

c_list = [0.1,1,10,100,1000]
test_accuracies = []
train_accuracies = []

for i in range(len(c_list)):
  #calculate Gaussian kernel #We need to calculate gaussian kernel only once since all of them uses same K matrix(same traning data points with different labels). 
  s = 10 #When s^2 is large, we get a more smooth function(underfit occurs). It is like the binwidth value h in the kernel smoother.
  K_train = gaussian_kernel(x_train, x_train, s)
  C = c_list[i];

  alpha_lst = []
  w0_lst = []
  y_train_c_lst = []
  for t in range(labels): #Evaluating alphas and w0 for each of the classes
    label = t + 1;
    y_train_c = ((y_train == label).astype(int))*2 - 1
    yyK = np.matmul(y_train_c[:,None], y_train_c[None,:]) * K_train 
    epsilon = 1e-3
    P = cvx.matrix(yyK)
    q = cvx.matrix(-np.ones((N_train, 1)))
    G = cvx.matrix(np.vstack((-np.eye(N_train), np.eye(N_train))))
    h = cvx.matrix(np.vstack((np.zeros((N_train, 1)), C * np.ones((N_train, 1)))))
    A = cvx.matrix(1.0 * y_train_c[None,:])
    b = cvx.matrix(0.0)
    result = cvx.solvers.qp(P, q, G, h, A, b)
    alpha = np.reshape(result["x"], N_train)
    alpha[alpha < C * epsilon] = 0
    alpha[alpha > C * (1 - epsilon)] = C
    support_indices, = np.where(alpha != 0)
    active_indices, = np.where(np.logical_and(alpha != 0, alpha < C))
    w0 = np.mean(y_train_c[active_indices] * (1 - np.matmul(yyK[np.ix_(active_indices, support_indices)], alpha[support_indices])))
    alpha_lst.append(alpha)
    w0_lst.append(w0)
    y_train_c_lst.append(y_train_c) 

  #Calculating the accuracy for training data
  train_predicted_lst = []
  for t in range(labels):
    train_predicted = np.matmul(K_train, y_train_c_lst[t][:,None] * alpha_lst[t][:,None]) + w0_lst[t]
    train_predicted_lst.append(train_predicted)
  train_predicted_matrix = np.hstack(train_predicted_lst)
  train_predictions = np.argmax(train_predicted_matrix,axis=1) + 1
  train_accuracies.append(accuracy_score(train_predictions,y_train)) #Adding to accuracy list
  
  #Calculating the accuracy for test data
  K_test = gaussian_kernel(x_test, x_train, s)
  test_predicted_lst = []
  for t in range(labels):
    test_predicted = np.matmul(K_test, y_train_c_lst[t][:,None] * alpha_lst[t][:,None]) + w0_lst[t] #4000x1
    test_predicted_lst.append(test_predicted)
  test_predicted_matrix = np.hstack([test_predicted_lst[i] for i in range(labels)]) #This returns 4000(N)x5(number of classes)
  test_predictions =  np.argmax(test_predicted_matrix,axis=1) + 1 #Since the output of argmax would range 0 to 4, we need to add 1
  test_accuracies.append(accuracy_score(test_predictions,y_test)) #Adding to accuracy list

     pcost       dcost       gap    pres   dres
 0: -1.3936e+02 -2.2047e+02  7e+03  3e+01  4e-15
 1: -3.4801e+01 -2.0264e+02  4e+02  9e-01  4e-15
 2: -2.8151e+01 -7.9601e+01  5e+01  2e-15  1e-15
 3: -3.1266e+01 -4.3423e+01  1e+01  7e-16  9e-16
 4: -3.2572e+01 -3.9120e+01  7e+00  6e-16  8e-16
 5: -3.3423e+01 -3.6637e+01  3e+00  2e-16  8e-16
 6: -3.3666e+01 -3.6029e+01  2e+00  2e-16  7e-16
 7: -3.4121e+01 -3.4971e+01  9e-01  5e-16  8e-16
 8: -3.4319e+01 -3.4585e+01  3e-01  2e-16  9e-16
 9: -3.4415e+01 -3.4445e+01  3e-02  4e-16  9e-16
10: -3.4429e+01 -3.4430e+01  1e-03  3e-16  9e-16
11: -3.4429e+01 -3.4429e+01  5e-05  7e-16  9e-16
12: -3.4429e+01 -3.4429e+01  1e-06  3e-16  9e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -1.1812e+02 -2.1835e+02  7e+03  3e+01  4e-15
 1: -3.1287e+01 -2.0015e+02  5e+02  1e+00  3e-15
 2: -2.4529e+01 -8.1514e+01  6e+01  2e-16  1e-15
 3: -2.7550e+01 -4.0807e+01  1e+01  3e-16  9e-16
 4: -2.8779e+01 -3.6453e+01  8e+00  2e-16  7e-1